In [2]:
# from tendo import singleton
# me = singleton.SingleInstance()
from food.tools import *
from food.paths import *
from food.psql import *

import pandas as pd
import numpy as np
from food.clipmodel import *


In [14]:

import os
os.environ["HF_DATASETS_OFFLINE"] = "2"
os.environ["TRANSFORMERS_OFFLINE"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from food.tools import *
from food.psql import *
from food.paths import *
from PIL import Image
from food.clipmodel import image2clip
import custom_pandas as cpd
from tqdm import tqdm
from sqlalchemy import update

query = """select id,description
             from foods
            where clip is Null """

total = engine.execute(f'select count(*) from ({query}) a').one()

bs = 1
pd_iter = cpd.read_sql_query(query, engine, chunksize=bs, index_col='id')


In [15]:

for inp in tqdm(pd_iter, desc="clip food inference", total=total[0] // bs):  
    clip = text2clip(inp['description'].iloc[0]).numpy().tolist()
    stmt = (update(Foods).
            values(clip=clip).
            where(Foods.id == int(inp.index[0])))
    engine.execute(stmt)



clip food inference: 0it [00:00, ?it/s]
